### training mode


In [49]:
import sys
from algorithm.module_list import *

In [5]:
url = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/news.csv"

df=pd.read_csv(url)
df=df.sample(frac=1).reset_index(drop=True)
df.head()

,title,category
0,Alaska Oil Spill Crews Seize on Calmer Weather...,Sci/Tech
1,PeopleSoft devotees in denial?,Sci/Tech
2,Blackhawks Re-Sign Defenseman Berard,Sports
3,Pakistani Militant Farooqi had Links With Top ...,World
4,Schaub Rallies Falcons,Sports


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [7]:
nltk.download('stopwords')
STOPWORDS=stopwords.words('english')
print(STOPWORDS[:5])
porter=PorterStemmer()

['i', 'me', 'my', 'myself', 'we']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erfan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def preprocess(text,stopwords=STOPWORDS):
    text=text.lower()
    pattern = re.compile(r"\b(" + r"|".join(stopwords) + r")\b\s*")
    text=pattern.sub('',text)
    text=re.sub(r"\([^)]*\)", "", text)

    
    text = re.sub(r"([-;;.,!?<=>])", r" \1 ", text)  # separate punctuation tied to words
    text = re.sub("[^A-Za-z0-9]+", " ", text)  # remove non alphanumeric chars
    text = re.sub(" +", " ", text)  # remove multiple spaces
    text = text.strip()
    return text

text='Grete week for the NUSE!'
preprocess(text)

'grete week nuse'

In [9]:
# Apply to dataframe
preprocessed_df = df.copy()
preprocessed_df.title = preprocessed_df.title.apply(preprocess)

In [30]:
import collections
TRAIN_SIZE=0.7
TEST_SIZE=0.3

def Split(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7)
    return X_train,X_test,y_train,y_test

X=preprocessed_df['title'].values
y=preprocessed_df['category'].values

X_train,X_test,y_train,y_test=Split(X,y)

print(f"X_train: {X_train.shape}\n"
        f"X_test: {X_test.shape}-> total-val: {len(preprocessed_df.title)}\n"
        f"y_train: {y_train.shape}\n"
        f"y_test: {y_test.shape}")



X_train: (84000,)
X_test: (36000,)-> total-val: 120000
y_train: (84000,)
y_test: (36000,)


####  Label encoding


In [62]:
from sklearn.preprocessing import LabelEncoder
CLASS_NUM=len(np.unique(y_train))
label_encoding1=OneHotEncoder().fit(y_train.reshape(-1,1))
label_encoding2=LabelEncoder().fit(y_train)

CLASS_LIST=list(label_encoding2.classes_)
y_train_enc=label_encoding2.transform(y_train)

In [45]:
del LabelEncoder

In [ ]:
from candlestick import candlestick
import pandas as pd
path=f'cryptoData/'
df=pd.read_csv(f'{path}ADA-USD.csv',parse_dates=True)
# df['result']= candlestick.inverted_hammer(df)
# print(df)
z= candlestick.hammer(df,ohlc=["Open", "High", "Low", "Close"],target='result')
z
# z['result_check']=z['result'].astype("category").cat.codes
# z = z[z['result_check'] != -1]

# z['result'].value_counts()
# z['result'].map(dict('True'=1, 'False'=0))
# type(z['result'][1])


In [ ]:
import plotly.graph_objects as go
from candlestick import candlestick
import pandas as pd
import matplotlib.pyplot as plt

# Define the Data
path=f'cryptoData/'
df=pd.read_csv(f'{path}SOL-USD.csv',parse_dates=True)

# Result of the bearish_engulfing candle
z= candlestick.dragonfly_doji(df,ohlc=["Open", "High", "Low", "Close"],target='result')
z['result_check']=z['result'].astype("category").cat.codes
z = z[z['result_check'] != -1]

# Create the candlestick chart
def plot(df,windows):

    fig = go.Figure(data=[go.Candlestick(x=df.loc[:,'Date'],
                  open=df.loc[:,'Open'], high=df.loc[:,'High'],low=df.loc[:,'Low'], close=df.loc[:,'Close'])])

    # fig.add_trace(go.Scatter(x=z.loc[windows:,'Date'], y=z.loc[windows:,'result_check'], name="linear",line_shape='linear'))

    fig.add_trace(go.Scatter(x=df.loc[windows:,'Date'], y=df.loc[windows:,'Close'],mode='markers',
                        marker=dict(size=8, color=[ 'green' if i ==1  else 'red' for i in df.loc[windows:,'result_check']], showscale=True)))
    
    fig.update_layout(width=1500, height=600)
    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.show()

# Show the chart
# show all the postive signal
b=z[z['result_check'] != 0]
df=b
plot(df,0)


In [211]:
b=z[z['result_check']]

b


In [ ]:

import torch
import gym
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split as split
from torch.utils.data import TensorDataset, DataLoader


# Load and preprocess the data
path=f'cryptoData/'
df=pd.read_csv(f'{path}ADA-USD.csv',parse_dates=True,index_col='Date')
data_YY=df['Close'].shift(30)
data_YY=torch.tensor(data_YY.fillna(data_YY.mean()))
data_XX=torch.tensor(df.values)

data =df.astype(float) # Convert the data to float type
data = (df -df.mean()) / df.std() # Normalize the data
data = torch.tensor(data.values).float() # Create a tensor from the data


window_size = 30
data_X = []
data_y = []
for i in range(len(data)-window_size):
    data_X.append(data_XX[i:window_size+i])
    data_y.append(data_YY[i:window_size+i])
    

data_X = torch.stack(data_X)
data_y = torch.stack(data_y)



# Define the model architecture
class FinancialModel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(FinancialModel, self).__init__()
        # self.rnn = torch.nn.GRU(input_size, 64, num_layers=2, batch_first=True)
        # self.fc = torch.nn.Linear(64, output_size)
        self.fc = torch.nn.Linear(6, 64)
        self.fc2 = torch.nn.Linear(64, 1)
        # self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x):
        # x, _ = self.rnn(x)
        # x = self.dropout(x[:, -1, :])
        x = self.fc(x)
        x=self.fc2(x)
        return x

X_train,X_test,y_train,y_test= split(data_X,data_y,test_size=0.2,shuffle=False)
model = FinancialModel(input_size=X_train.shape[1], output_size=30)


train_data = TensorDataset(X_train, y_train)
train_loader=DataLoader(train_data,shuffle=False)


# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.MSELoss()

# Define the number of training iterations
num_iterations = 40

for iteration in range(num_iterations):
    Loss=[]
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.squeeze().float())
    
        # print(output.squeeze().shape)
        # print(y_batch.abssqueeze().shape)
        plt.ion()
        loss = loss_fn(output.squeeze().float(), y_batch.squeeze().float())
        Loss.append(loss)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            plt.clf() # clear the plot
            plt.plot(np.arange(len(Loss)), Loss,c='red')
            plt.show()
            plt.pause(0.001) # pause for a short time




In [194]:
X_batch.shape

torch.Size([1, 30, 6])

In [332]:
a = np.random.choice([0, 1], size=10, p=[0.5, 0.5])
X = np.random.rand(10)
def counter():
    a_plus=[]
    box=[]
    for index,z in enumerate(a):
        if z ==1:  
            box.append(0)
            if len(box) <2:
                first_index=index
            
            elif len(box) >1:                
                last_index=index
                box.clear()
                a_plus.append(X[first_index:last_index+1])
                a_plus.append('hii')
    
    return a_plus
a_plus=counter()
a_plus

[array([0.42317942, 0.60885058]),
 'hii',
 array([0.19092309, 0.31636793]),
 'hii',
 array([0.53457839, 0.60410112]),
 'hii']

(array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0]),
 array([0.20614717, 0.79801224, 0.42317942, 0.60885058, 0.19092309,
        0.31636793, 0.53457839, 0.60410112, 0.78405289, 0.04281032]))

In [ ]:
[0.92448713, 0.65885908, 0.59642019, 0.54274564, 0.39097281,],
0.96784113, 0.93806501, 0.85367407, 0.19426317


In [ ]:
# a = np.random.choice([0.1,0.2], size=10, p=[0.6, 0.4])
a = np.random.rand(5)
a